In [1]:
#Code for dummy disease map. Generates lung mask with random values corresponding to diseases.
import numpy as np

# Define the dimensions of the 3D array
dimensions = (200, 200, 200)

# Define the values and their corresponding probabilities
values = [0, 2, 3, 4]
probabilities = [0.1, 0.7, 0.1, 0.1]

# Create the 3D array with random values based on probabilities
random_values = np.random.choice(values, size=dimensions, p=probabilities)

# Now, 'random_values' is a 3D NumPy array with the specified dimensions and values sampled based on probabilities.

In [2]:
#Function from Nate's code
def update_camera_info(obj, evt):
                    camera = mlab.gcf().scene.camera
                    print("Camera Position:", camera.position)
                    print("Camera Focal Point:", camera.focal_point)
                    print("Camera View Up:", camera.view_up)

In [3]:
#Load in the file
import nrrd
import nibabel as nib
from mayavi import mlab

# Load the NIfTI image using nibabel
file_path = r'Z:\D-Images\SPIROMICS-SubStudy\2-Results-CheckedDoneTemp\1-Done\Case-50-Spiromics-45626090\seg_net\SPI-IA210305-PBV_FRC_0.75_Qr40d_VNC.mask.nii.gz'
nii_image = nib.load(file_path)

In [47]:
# Get the image data as a NumPy array
image_data = nii_image.get_fdata().astype(int)
condition = image_data > 0
image_data[condition] = 1

#Center of mass coordinates
coords = np.argwhere(image_data)
center_of_mass = np.mean(coords, axis=0)
center_of_mass = center_of_mass.astype(int)   #Remove /10 when finished downsampling
image_data[*center_of_mass] = 2

In [39]:
for i in range(3):
    # Define the dimensions of the array
    shape = (512, 512, 556)

    # Define the center and cutoff distance
    if i == 0:
        center = (340, 350, 320)
    elif i == 1:
        center = (300, 175, 220)
    elif i == 2:
        center = (165, 235, 300)

    cutoff_distance = 25

    # Create coordinate grids for each dimension
    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))

    # Calculate the Euclidean distance from the center for each point
    distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)

    # Apply the Gaussian distribution with a mean of 0 and standard deviation of 1
    gaussian = np.exp(-0.5 * (distance / cutoff_distance)**2)

    if i == 0:
        g1 = gaussian.copy()
        g1 = (g1*image_data)#[::10,::10,::10]
    if i == 1:
        g2 = gaussian.copy()
        g2 = (g2*image_data)#[::10,::10,::10]
    if i == 2:
        g3 = gaussian.copy()
        g3 = (g3*image_data)#[::10,::10,::10]

gaussians = g1 + g2 + g3

In [51]:
# Function for creating sample disease map. Takes in three centers and the shape of the object and creates three gaussian distributions at those centers that models
# a disease. Returns a 3D numPy array containing the three gaussian plots that represents a disease volume.
def create_three_gaussian_disease(center1, center2, center3, shape):
    for i in range(3):
        # Define the dimensions of the array
        shape = shape

        # Define the center and cutoff distance
        if i == 0:
            center = center1
        elif i == 1:
            center = center2
        elif i == 2:
            center = center3

        cutoff_distance = 25

        # Create coordinate grids for each dimension
        x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))

        # Calculate the Euclidean distance from the center for each point
        distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)

        # Apply the Gaussian distribution with a mean of 0 and standard deviation of 1
        gaussian = np.exp(-0.5 * (distance / cutoff_distance)**2)

        if i == 0:
            g1 = gaussian.copy()
            g1 = (g1*image_data)#[::10,::10,::10]
        if i == 1:
            g2 = gaussian.copy()
            g2 = (g2*image_data)#[::10,::10,::10]
        if i == 2:
            g3 = gaussian.copy()
            g3 = (g3*image_data)#[::10,::10,::10]

    gaussians = g1 + g2 + g3
    return gaussians

In [65]:
# Function for visualizing centers on a lung object. Used for making sample gaussian disease distributions.
def visualizeCenter(center, object, shape):
#Create single gaussian
    # Define the center and cutoff distance
    cutoff_distance = 25
    # Create coordinate grids for each dimension
    x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    # Calculate the Euclidean distance from the center for each point
    distance = np.sqrt((x - center[0])**2 + (y - center[1])**2 + (z - center[2])**2)
    # Apply the Gaussian distribution with a mean of 0 and standard deviation of 1
    gaussian = np.exp(-0.5 * (distance / cutoff_distance)**2)
    gaussian = (gaussian*object)[::10,::10,::10]

#Visualize gaussian on object
    # Create a figure
    mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white
    # Visualize the entire image as a volume (adjust opacity as needed)
    volume1 = mlab.pipeline.volume(mlab.pipeline.scalar_field(gaussian),color=(1,0,0))
    volume1._volume_property.scalar_opacity_unit_distance = 1.0   #Appears to be affecting spread of gaussians??

    lungs = mlab.contour3d(object[::10,::10,::10], contours=[0.5], opacity = 0.15, color = (128/255, 174/255, 128/255))  #Include [::10,::10,::10] to downsample

    mlab.view(azimuth=90, elevation=90, distance=(600/0.546875)/10, focalpoint=center_of_mass/10)
    mlab.gcf().scene.camera.view_angle = 23
    mlab.show()
        

In [33]:
gaussians.max()

1.0000000000000027

In [6]:
import numpy as np
import nrrd

# Create a 3D NumPy array (example)
my_3d_array = gaussians

# Define the file path for saving the NRRD file
file_path = "output1.nrrd"

# Save the 3D NumPy array as an NRRD file
nrrd.write(file_path, my_3d_array)

print("NRRD file saved successfully.")

NRRD file saved successfully.


In [52]:
#Create sample disease map 1
disease1 = create_three_gaussian_disease((340, 350, 320), (300, 175, 220), (165, 235, 300), (512, 512, 556))

In [54]:
#Create sample disease map 2
disease2 = create_three_gaussian_disease((120, 350, 280), (239, 275, 300), (140, 200, 340), (512, 512, 556))

In [67]:
#Create sample disease map 3
visualizeCenter((239, 275, 300), image_data, (512, 512, 556))

KeyboardInterrupt: 

In [55]:
# Create a figure
mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white

# Visualize the entire image as a volume (adjust opacity as needed)
volume1 = mlab.pipeline.volume(mlab.pipeline.scalar_field(disease1),color=(1,0,0))
volume1._volume_property.scalar_opacity_unit_distance = 1.0   #Appears to be affecting spread of gaussians??

volume2 = mlab.pipeline.volume(mlab.pipeline.scalar_field(disease2),color=(128/255, 0/255, 128/255))
volume2._volume_property.scalar_opacity_unit_distance = 1.0

lungs = mlab.contour3d(image_data, contours=[0.5], opacity = 0.15, color = (128/255, 174/255, 128/255))  #Include [::10,::10,::10] to downsample
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

mlab.view(azimuth=90, elevation=90, distance=(600/0.546875), focalpoint=center_of_mass)
mlab.gcf().scene.camera.view_angle = 23
mlab.show()

Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]
Camera Position: [ 249.         1377.14285714  303.        ]
Camera Focal Point: [249. 280. 303.]
Camera View Up: [0. 0. 1.]


In [8]:
# Create a figure
mlab.figure(bgcolor=(1, 1, 1))  # Set the background color to white

# Visualize the entire image as a volume (adjust opacity as needed)
lungs = mlab.contour3d(image_data[::10,::10,::10], contours=[0.5], opacity = 0.15, color = (128/255, 174/255, 128/255))
mlab.gcf().scene.interactor.add_observer('MouseMoveEvent', update_camera_info)

#Dummy Lung Mask
obj = (array == 2).astype(int)
mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (1, 0, 0))   #Remove later
# obj = (diseaseEX == 3).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (0, 0, 1))
# obj = (diseaseEX == 4).astype(int)
# mlab.contour3d(obj[::10,::10,::10], contours=[0.5], opacity = 0.9, color = (148/255, 0, 211/255))

# Set the camera position and direction to focus on the center of mass
mlab.view(azimuth=90, elevation=90, distance=600/0.546875, focalpoint=(0,0,0))
mlab.gcf().scene.camera.view_angle = 23

#Saving file
outputPath = r"z:/Q- Figures/T32-Poster/Nick/Experiment/"
mlab.savefig(outputPath + "test1.png")   #, magnification=5     <-- Add argument for magnification factor

# Show the visualization
mlab.show()

Exception occurred in traits notification handler for object: <mayavi.tools.modules.IsoSurfaceFactory object at 0x00000240AC042390>, trait: contours, old value: 5, new value: [0.5]
Traceback (most recent call last):
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_notifiers.py", line 342, in __call__
    self.handler(*args)
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\mayavi\tools\modules.py", line 185, in _contours_changed
    self._target.contour.contours = self.contours
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_types.py", line 2695, in validate
    return TraitListObject(self, object, name, value)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nichmoore\Documents\GitHub\RA4-Project\venvnick\Lib\site-packages\traits\trait_list_object.py", line 584, in __init__
    super().__init__(
  File "

TraitError: Each element of the 'contours' trait of a Contour instance must be 0.0 <= a number <= 0.0, but a value of 0.5 <class 'float'> was specified.